In [1]:
########## HIDE CODE BLOCKS ##########

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
########## INITIALIZATION ##########
from scripts.Beer_Advisor_Helper import *
import warnings
warnings.filterwarnings('ignore')

########## PRESENTATION MAIN ##########
display(show_overview_btn)
show_overview_btn.on_click(show_overview_text)
show_webscraping_btn.on_click(show_webscraping_text)
show_EDA_btn.on_click(show_EDA1_text)
show_boxhist_btn.on_click(goplot_boxhist)
show_superuserplot_btn.on_click(goplot_super_user)
show_correlation_btn.on_click(goplot_correlation)
show_beeradvisor1_btn.on_click(show_beeradvisor1_text)
show_beeradvisor2_btn.on_click(show_beeradvisor2_text)
show_beeradvisor3_btn.on_click(show_beeradvisor3_text)
show_beeradvisor4_btn.on_click(show_beeradvisor4_text)
show_beeradvisor5_btn.on_click(show_beeradvisor5_text)
show_beerrecs_btn.on_click(show_beerrecs)
show_shameless_plug_btn.on_click(show_shameless_plug)



All Scripts Imported!!!


Button(button_style='primary', description="Let's get started!", style=ButtonStyle(), tooltip='Show Overview B…